In [25]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

In [26]:
#lectura del dataset
df = pd.read_csv('./input/simpsons_dataset.csv')
df.shape
df.head()


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [27]:
#removemos las filas donde no hay dialogos
df.isnull().sum()
df = df.dropna().reset_index(drop=True)
df.isnull().sum()

raw_character_text    0
spoken_words          0
dtype: int64

In [28]:
#limpieza del dataset 
#para eso usamos spacy 
"""
Free open-source library for NLP.
It features NER, POS tagging, dependency parsing, word vectors and more.
"""
# descargar el paquete en inglés ----> python -m spacy  download en
nlp = spacy.load('en', disable=['ner', 'parser']) # se desabilita ner y parser


In [29]:
# removemos caracteres no alfabeticos 
#creamos un generador que va  a sustituir caracter no alfabeticos con espacios

def replace_chars(df, column, pattern, replacement):
    text = []
    for row in df[ column]:
        row = re.sub(pattern, replacement, str(row)).lower()
        text.append(row)
    return text

pattern = "[^A-Za-z']+"
column = 'spoken_words'
text_only_alpha = replace_chars(df, column, pattern, ' ')

In [30]:

#crear objetos tipo doc de spacy usando un generador
    #spicy funciona ve a los textos como  streams  enviandolos por tuberias(pipes)
docs_gen = nlp.pipe(text_only_alpha, batch_size=5000, n_threads=-1)

#los pasamos a una lista
sp_docs = [] 
for sp_doc in docs_gen:
    sp_docs.append(sp_doc)

In [31]:
#removemos stop words y lemmatizamos
def remove_stop_words_lemmatize(sp_docs):
    clean_docs = []
    for doc in sp_docs:
        clean_doc = []
        for token in doc:
            if not token.is_stop:
                clean_doc.append(token.lemma_ )
        if len(clean_doc) > 2: #los textos con solo dos tokens no valen la pena
            clean_docs.append( ' '.join(clean_doc) ) 
    return clean_docs

clean_docs = remove_stop_words_lemmatize(sp_docs)

In [32]:
from gensim.models.phrases import Phrases, Phraser
sent = [row.split() for row in clean_docs]


print(clean_docs)

phrases = Phrases(sent, min_count=30, progress_per=10000)

bigram = Phraser(phrases)
sentences = bigram[sent]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
import multiprocessing

from gensim.models import Word2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer


w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [34]:
w2v_model.build_vocab(sentences, progress_per=10000)
print(w2v_model)

Word2Vec(vocab=3340, size=300, alpha=0.03)


In [35]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(6021032, 15803190)

In [36]:
w2v_model.save("word2vec_simpsons.model")

In [37]:
#tests

print ( w2v_model.wv.similarity("moe", 'tavern') ) 
print ( w2v_model.wv.similarity('bart', 'nelson') )

print ( w2v_model.wv.doesnt_match(["nelson", "bart", "milhouse"]) ) 
print ( w2v_model.wv.doesnt_match(['homer', 'patty', 'selma']) ) 



0.89936185
0.6753222
nelson
homer


/home/javier/anaconda3/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
